# IS 733 Data Mining (Fall 2022)
### Homework 1
### Data Wrangling with Baltimore City 911 Report Data
### by Nadeem Ahmed

#### **Question 1**

We start by importing relevant libraries and attempt to summarize the data set before pre-processing. We will also exclude certain columns as they are either constants or have limited value.  

**Note 1: Trying to do profiling using Pandas Profiling on this dataset proved to be not fruitful as the library was not able to handle some elements of the dataset and the error produced was "Product space too large to allocate arrays!"**  

**Note 2: Cells need to be run in sequence as some variables are reused.**

In [ ]:
!pip install skimpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import plotly.express as px # usig the Plotly plotting engine
from ipywidgets import interact, widgets # For interactive dashboard
from skimpy import skim # A lite library to summarize a dataframe quickly

In [ ]:
columns_excluded = ['Post','Total Incidents','Location 1','vri_name1'] # exclude columns that dont provide value

df = pd.read_csv("https://ruralatlasdataset.s3.amazonaws.com/BPD_Part_1_Victim_Based_Crime_Data.csv", usecols=lambda column: column not in columns_excluded)

#Convert certain columns to categories
df["District"] = df["District"].astype("category")
df["Description"] = df["Description"].astype("category")
df["Weapon"] = df["Weapon"].astype("category")

We will answer the following bullets by running **skimpy**:
+ Total number of instances/rows
+ Total number of features/columns
+ For each column
    + Are they numeric/real/continuous or symbolic/discrete types of attributes or belong to temporal or spatial categories
    + If it is a numeric attribute, what are the min, max, mean, median, and standard deviation of the values
    + If it is a discrete attribute, what is the total number of unique values?
    + What is the level of missingness (% of rows with missing values)?

In [ ]:
skim(df)

╭───────────────────────────────────── skimpy summary ──────────────────────────────────────╮
│          Data Summary                Data Types               Categories                  │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓ ┏━━━━━━━━━━━━━━━━━━━━━━━┓          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃ ┃ Categorical Variables ┃          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩ ┡━━━━━━━━━━━━━━━━━━━━━━━┩          │
│ │ Number of rows    │ 292761 │ │ object      │ 7     │ │ Description           │          │
│ │ Number of columns │ 12     │ │ category    │ 3     │ │ Weapon                │          │
│ └───────────────────┴────────┘ │ float64     │ 2     │ │ District              │          │
│                                └─────────────┴───────┘ └───────────────────────┘          │
│                                          number                                           │
│ ┏━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━┳━━━━━┳━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃           ┃ missing ┃ complete rate ┃ mean ┃ sd    ┃ p0  ┃ p25 ┃ p75 ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━╇━━━━━╇━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ Longitude │    1600 │          0.99 │  -77 │ 0.044 │ -82 │ -77 │ -77 │  -76 │      █ │  │
│ │ Latitude  │    1600 │          0.99 │   39 │  0.03 │  38 │  39 │  39 │   40 │     █▅ │  │
│ └───────────┴─────────┴───────────────┴──────┴───────┴─────┴─────┴─────┴──────┴────────┘  │
│                                         category                                          │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┓  │
│ ┃                    ┃ missing      ┃ complete rate         ┃ ordered      ┃ unique    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━┩  │
│ │ Description        │            0 │                     1 │ False        │        14 │  │
│ │ Weapon             │       230000 │                  0.21 │ False        │         6 │  │
│ │ District           │            0 │                     1 │ False        │        10 │  │
│ └────────────────────┴──────────────┴───────────────────────┴──────────────┴───────────┘  │
╰─────────────────────────────────────────── End ───────────────────────────────────────────╯

+ What are the three attribute values with the largest count?  

We can look at the three categorical variables to get that answer.

In [ ]:
df['Description'].value_counts().head(3)

LARCENY           65508
COMMON ASSAULT    48061
BURGLARY          42237
Name: Description, dtype: int64

In [ ]:
df['District'].value_counts().head(3)

NORTHEAST    44207
SOUTHEAST    41816
CENTRAL      33508
Name: District, dtype: int64

In [ ]:
df['Weapon'].value_counts().head(3)

FIREARM    29036
OTHER      17313
KNIFE      10023
Name: Weapon, dtype: int64

#### **Question 2:**

Generate a series of plots to describe the temporal pattern (year-to-year, monthly, and day-of-week) of the overall crime incidence aggregating from all geo-locations.

##### Data Munging
We will do the following pre-processing of data to increase our ability of filtering and plotting the data.  

+ Lets add another **"Date"** column to convert **"CrimeDate"** and the **"CrimeTime"** as a datetime object so that we can access its date/time attributes easily.
+ We will remove rows of years in which there is hardly any data. For our purposes, the data to be plotted will only be from 2014-2019. Rest of the years will be removed.

In [ ]:
# Crimetime column has quiet a few empty cells. So we have to fill them up before we convert. Otherwise does not work.

df = df.fillna({
    'CrimeTime' : '0:00:00'
})

# This takes about 20+ secs to execute. There are various other ways to this but ....

df['Date'] = pd.to_datetime(df['CrimeDate'] + ' ' + df['CrimeTime'])

In [ ]:
# Remove rows before 2014 & after 2019
df = df[~((df['Date'] < '2014-01-01') | (df['Date'] > '2019-12-31'))]

In [ ]:
# Perform group by operations for Year, Month and Weekday
df_year = df.groupby(df['Date'].dt.year)
df_month = df.groupby(df['Date'].dt.month)
df_week = df.groupby(df['Date'].dt.weekday)

In [ ]:
# Gives us a Series object with years aggregated across the years
crimes_by_years = df_year.size()
crimes_by_years

Date
2014    45307
2015    48192
2016    48774
2017    52163
2018    48436
2019    45901
dtype: int64

In [ ]:
# Gives us a Series object with months aggregated across the years
crimes_by_month = df_month.size()

# For plotting purposes we will need to rename the index of the Series objects to actual Months and Days
months = { 1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
crimes_by_month.rename(index=months, inplace=True)
crimes_by_month

Date
Jan    22569
Feb    18295
Mar    21036
Apr    23061
May    25684
Jun    25741
Jul    26352
Aug    26568
Sep    25635
Oct    26391
Nov    23917
Dec    23524
dtype: int64

In [ ]:
# Gives us a Series object with days aggregated across the years
crimes_by_days = df_week.size()

# For plotting purposes we will need to rename the index of the Series objects to actual Months and Days
days_of_week = {0:'Mon',1:'Tue',2:'Wed',3:'Thu',4:'Fri',5:'Sat',6:'Sun'}
crimes_by_days.rename(index=days_of_week, inplace=True)
crimes_by_days

Date
Mon    42493
Tue    41708
Wed    41466
Thu    41183
Fri    43024
Sat    40035
Sun    38864
dtype: int64

In [ ]:
#Plot crime aggregate across Years
fig_year = px.line(crimes_by_years, title="Crime aggregate across years")
fig_year.update_layout(showlegend=False)
fig_year.show()

In [ ]:
#Plot crime aggregate across Months
fig_month = px.line(crimes_by_month, title="Crime aggregate across months")
fig_month.update_layout(showlegend=False)
fig_month.show()

In [ ]:
#Plot crime aggregate across Days
fig_days = px.line(crimes_by_days,title="Crime aggregate across days")
fig_days.update_layout(showlegend=False)
fig_days.show()

#### **Question 3**

Generate a plot describing the distribution of crime type aggregating from all geo-locations and all time periods.

In [ ]:
#Generate an array to capture the Crime Types against Years
crime_series = []
for year in df_year.groups.keys():
    temp_group = df_year.get_group(year)
    temp_group = temp_group['Description'].value_counts()
    temp_group.rename(year, inplace=True)
    crime_series.append(temp_group)

In [ ]:
crimes = pd.DataFrame()

for crime in crime_series:
    crimes = pd.concat([crimes,crime],axis=1)

crimes = crimes.T
crimes.rename_axis('Crimes',axis='columns', inplace=True)
crimes.rename_axis('Years',axis='rows',inplace=True)
crimes

Crimes,AGG. ASSAULT,ARSON,AUTO THEFT,BURGLARY,COMMON ASSAULT,HOMICIDE,LARCENY,LARCENY FROM AUTO,RAPE,ROBBERY - CARJACKING,ROBBERY - COMMERCIAL,ROBBERY - RESIDENCE,ROBBERY - STREET,SHOOTING
Years,,,,,,,,,,,,,,
2014,4264,216,3652,6877,7463,211,11382,6719,241,156,627,468,2662,369
2015,4763,289,4565,7856,6992,342,10671,7057,283,287,883,439,3133,632
2016,5149,268,4612,7392,7498,318,10459,6517,289,413,865,529,3799,666
2017,5843,265,4669,8094,8751,342,10789,6176,362,568,1078,559,3964,703
2018,5604,126,4226,6212,8369,309,10693,6368,349,481,913,513,3596,677
2019,5642,106,3760,5381,8222,347,10618,5679,272,581,753,414,3359,767


In [ ]:
# Aggregates of Crime Types across years

fig_crime = px.line(crimes, title="Crime Types against Years")
fig_crime.show()

In [ ]:
#Alternate representation using bar chart

fig_bar = px.bar(crimes, title="Crime Types against Years")
fig_bar.show()

#### **Question 4**

Generate a series of plots to illustrate how crime types distributions might vary from year-to-year, month-to-month, or by day-of-week aggregating across all locations

**NOTE:** The by the year crimes plot is already given above. Below are the plots for the by the month and by the week day distributions.  

In [ ]:
# Generate an array to capture the Crime Types against Mnnths

crime_series = []
for month in df_month.groups.keys():
    temp_group = df_month.get_group(month)
    temp_group = temp_group['Description'].value_counts()
    temp_group.rename(month, inplace=True)
    crime_series.append(temp_group)

In [ ]:
crimes = pd.DataFrame()

for crime in crime_series:
    crimes = pd.concat([crimes,crime],axis=1)

crimes = crimes.T
crimes.rename(index=months, inplace=True)
crimes.rename_axis('Crimes',axis='columns', inplace=True)
crimes.rename_axis('Months',axis='rows',inplace=True)
crimes

Crimes,AGG. ASSAULT,ARSON,AUTO THEFT,BURGLARY,COMMON ASSAULT,HOMICIDE,LARCENY,LARCENY FROM AUTO,RAPE,ROBBERY - CARJACKING,ROBBERY - COMMERCIAL,ROBBERY - RESIDENCE,ROBBERY - STREET,SHOOTING
Months,,,,,,,,,,,,,,
Jan,2217,86,2367,3463,3518,147,4840,2951,139,225,495,265,1605,251
Feb,1865,71,1711,2683,3330,103,4124,2236,128,148,313,174,1243,166
Mar,2414,88,1812,2836,3956,108,4822,2574,162,148,374,239,1267,236
Apr,2758,141,1916,3382,4115,147,5214,2818,172,183,318,199,1413,285
May,3017,132,2081,3528,4374,180,5881,3209,166,172,424,265,1869,386
Jun,2937,119,2275,3631,4261,155,5794,3479,172,193,453,244,1650,378
Jul,2966,123,2424,3740,3972,204,5883,3617,157,263,457,251,1882,413
Aug,2956,102,2302,3671,3965,179,6008,3828,161,251,413,301,1996,435
Sep,2778,108,2099,3747,4191,173,5821,3467,142,189,417,209,1955,339


In [ ]:
# Aggregates of Crime Types across months

fig_crime_months = px.line(crimes, title="Crime Types against Months")
fig_crime_months.show()

In [ ]:
#Alternate representation using bar chart

fig_crime_months_bar = px.bar(crimes, title="Crime Types against Months")
fig_crime_months_bar.show()

In [ ]:
# Generate an array to capture the Crime Types against Days

crime_series = []
for day in df_week.groups.keys():
    temp_group = df_week.get_group(day)
    temp_group = temp_group['Description'].value_counts()
    temp_group.rename(day, inplace=True)
    crime_series.append(temp_group)

crimes = pd.DataFrame()

for crime in crime_series:
    crimes = pd.concat([crimes,crime],axis=1)

crimes = crimes.T
crimes.rename(index=days_of_week, inplace=True)
crimes.rename_axis('Crimes',axis='columns', inplace=True)
crimes.rename_axis('Days',axis='rows',inplace=True)

In [ ]:
# Aggregates of Crime Types across days

fig_crime_days = px.line(crimes, title="Crime Types against Days")
fig_crime_days.show()

In [ ]:
#Alternate representation using bar chart

fig_crime_days_bar = px.bar(crimes, title="Crime Types against Days")
fig_crime_days_bar.show()

#### **Question 5**

Design a dashboard that allows users like Gary to explore the spatial and temporal patterns of crime. You may get inspiration from tasks 2-4, but feel free to add additional bringing in additional insights. **Bonus: Create a real dynamic dashboard that allows users to change parameters such as location and time period.**

In [ ]:
# Lets add another column which captures just the Years. This will be used for animation slider.

df['Year'] = df['Date'].dt.year

# Now lets group against the Crime Types

df_crimes = df.groupby('Description')

In [ ]:
# We need to capture all the grouped dataframes of crime types in a dictionary so that we can use it in the interactive plotting

crime_types = {}
for crime_type in df_crimes.groups.keys():
    crime_t = df_crimes.get_group(crime_type)
    crime_t_sorted = crime_t.sort_values('Year', ascending=True)
    crime_types.update({crime_type:crime_t_sorted})

In [ ]:
crime_types.keys()

dict_keys(['AGG. ASSAULT', 'ARSON', 'AUTO THEFT', 'BURGLARY', 'COMMON ASSAULT', 'HOMICIDE', 'LARCENY', 'LARCENY FROM AUTO', 'RAPE', 'ROBBERY - CARJACKING', 'ROBBERY - COMMERCIAL', 'ROBBERY - RESIDENCE', 'ROBBERY - STREET', 'SHOOTING'])

In [ ]:
# Sample of the ARSON crime type across years. This can be changed to any of the values from the above dictionary
crime_types.get('ARSON')

,CrimeDate,CrimeTime,CrimeCode,Location,Description,Inside_Outside,Weapon,District,Neighborhood,Longitude,Latitude,Premise,Date,Year
292135,01/06/2014,15:16:00,8FO,1300 W 36TH ST,ARSON,I,FIRE,NORTHERN,HAMPDEN,-76.636488,39.330646,SCHOOL,2014-01-06 15:16:00,2014
263341,08/28/2014,16:37:00,8AO,300 E UNIVERSITY PW,ARSON,I,FIRE,NORTHERN,OAKENSHAWE,-76.613518,39.329917,APT/CONDO - OCCUPIED,2014-08-28 16:37:00,2014
263205,08/29/2014,13:49:00,8H,5000 E HOFFMAN ST,ARSON,O,FIRE,NORTHEAST,ARMISTEAD GARDENS,-76.556377,39.307609,ALLEY,2014-08-29 13:49:00,2014
261904,09/08/2014,2:28:00,8H,500 KINGSTON RD,ARSON,O,FIRE,SOUTHWEST,WESTGATE,-76.709514,39.286582,STREET,2014-09-08 02:28:00,2014
261869,09/08/2014,17:20:00,8J,1600 WILKENS AV,ARSON,O,FIRE,SOUTHERN,NEW SOUTHWEST/MOUNT CLARE,-76.642350,39.283596,YARD,2014-09-08 17:20:00,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35321,05/05/2019,5:02:00,8H,3300 SPAULDING AVE,ARSON,O,FIRE,NORTHWEST,CENTRAL PARK HEIGHTS,-76.676148,39.347367,STREET,2019-05-05 05:02:00,2019
36143,04/28/2019,1:30:00,8BO,2900 EDMONDSON AVE,ARSON,O,FIRE,SOUTHWEST,FRANKLINTOWN ROAD,-76.665103,39.294536,STREET,2019-04-28 01:30:00,2019
36144,04/28/2019,1:30:00,8BO,2900 EDMONDSON AVE,ARSON,NaN,FIRE,SOUTHWEST,FRANKLINTOWN ROAD,-76.665103,39.294536,NaN,2019-04-28 01:30:00,2019
36521,04/26/2019,10:50:00,8J,1700 E 25TH ST,ARSON,I,FIRE,EASTERN,DARLEY PARK,-76.592970,39.315781,ROW/TOWNHOUSE-OCC,2019-04-26 10:50:00,2019


#### **Interactive Dashboard Instructions** : Cell below output shows a drop down of all crime types. By selecting different drop down options, the plot can be changed to that crime type. After changing the crime type, run the next cell again to see the dashboard change for the crime type selected.

In [ ]:
# Now we will use the ipy widget for selection to populate a selection control with the crime types for options to be selected

crime_dropD = widgets.Dropdown(
    options=[crime_type for crime_type in crime_types.keys()],
    value='AGG. ASSAULT',
    description='Select Crime Type',
    disabled=False,
    )

# Display the control
display(crime_dropD)

Dropdown(description='Select Crime Type', options=('AGG. ASSAULT', 'ARSON', 'AUTO THEFT', 'BURGLARY', 'COMMON …

#### **Run the below cell again after changing the option in the drop down above. Press the "Play" button at the bottom of the map to see an animation of the crime densities across the years. There is also a slider which can be moved to show the temporal and geographical patters. Map can be moved around and zoomed in and out.**

In [ ]:
# This code block generates a mapbox using the Plotly function for density map box with an interactive animation slider and play button for animation

mapbox_title = "Crime Types Temporal and Spatial Distributions - " + crime_dropD.value

fig_crimes_geo = px.density_mapbox(crime_types.get(crime_dropD.value), lat='Latitude', lon='Longitude', radius=10,
                        center=dict(lat=39.299236, lon=-76.609383), zoom=11, title=mapbox_title,
                        mapbox_style="open-street-map", height=900, width=1050,animation_frame="Year", hover_name="Neighborhood")
fig_crimes_geo.show()